<a href="https://colab.research.google.com/github/Eboselethefirst/Mini_projects/blob/main/Clustering_With_SparkML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install findspark

In [2]:
def warn(*args,**kwargs):
  pass
import warnings as wng
wng.warn = warn
wng.filterwarnings("ignore")

In [3]:
import findspark as fs
fs.init()

from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans

In [5]:
spark = SparkSession.builder.appName("Clustering Using SparkML").getOrCreate()

In [6]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/customers.csv
!ls

--2026-01-21 14:07:04--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/customers.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8909 (8.7K) [text/csv]
Saving to: ‘customers.csv’

customers.csv       100%[===================>]   8.70K  --.-KB/s    in 0s      

2026-01-21 14:07:05 (201 MB/s) - ‘customers.csv’ saved [8909/8909]

customers.csv  sample_data


In [7]:
df = spark.read.csv("customers.csv",header=True,inferSchema=True)

In [9]:
df.printSchema()

root
 |-- Fresh_Food: integer (nullable = true)
 |-- Milk: integer (nullable = true)
 |-- Grocery: integer (nullable = true)
 |-- Frozen_Food: integer (nullable = true)



In [12]:
df.show(n=5, truncate=False)

+----------+----+-------+-----------+
|Fresh_Food|Milk|Grocery|Frozen_Food|
+----------+----+-------+-----------+
|12669     |9656|7561   |214        |
|7057      |9810|9568   |1762       |
|6353      |8808|7684   |2405       |
|13265     |1196|4221   |6404       |
|22615     |5410|7198   |3915       |
+----------+----+-------+-----------+
only showing top 5 rows


In [17]:
assembler = VectorAssembler(inputCols=["Fresh_Food","Milk","Grocery","Frozen_Food"], outputCol="features")
df_new = assembler.transform(df)

In [18]:
kmeans = KMeans(k=3)


In [19]:
model =kmeans.fit(df_new)

In [20]:
predictions = model.transform(df_new)

In [21]:
predictions.show()

+----------+-----+-------+-----------+--------------------+----------+
|Fresh_Food| Milk|Grocery|Frozen_Food|            features|prediction|
+----------+-----+-------+-----------+--------------------+----------+
|     12669| 9656|   7561|        214|[12669.0,9656.0,7...|         0|
|      7057| 9810|   9568|       1762|[7057.0,9810.0,95...|         0|
|      6353| 8808|   7684|       2405|[6353.0,8808.0,76...|         0|
|     13265| 1196|   4221|       6404|[13265.0,1196.0,4...|         0|
|     22615| 5410|   7198|       3915|[22615.0,5410.0,7...|         2|
|      9413| 8259|   5126|        666|[9413.0,8259.0,51...|         0|
|     12126| 3199|   6975|        480|[12126.0,3199.0,6...|         0|
|      7579| 4956|   9426|       1669|[7579.0,4956.0,94...|         0|
|      5963| 3648|   6192|        425|[5963.0,3648.0,61...|         0|
|      6006|11093|  18881|       1159|[6006.0,11093.0,1...|         1|
|      3366| 5403|  12974|       4400|[3366.0,5403.0,12...|         0|
|     

In [22]:
predictions.groupby("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   49|
|         2|   60|
|         0|  331|
+----------+-----+



In [23]:
predictions.createTempView("customers")

In [33]:
spark.sql("Select prediction, count(prediction) from customers GROUP BY prediction").show()

+----------+-----------------+
|prediction|count(prediction)|
+----------+-----------------+
|         1|               49|
|         2|               60|
|         0|              331|
+----------+-----------------+



In [35]:
spark.stop()

In [36]:
spark = SparkSession.builder.appName("Seed Clustering").getOrCreate()

In [37]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/seeds.csv

--2026-01-21 14:27:57--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/seeds.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8973 (8.8K) [text/csv]
Saving to: ‘seeds.csv’

seeds.csv           100%[===================>]   8.76K  --.-KB/s    in 0s      

2026-01-21 14:27:57 (180 MB/s) - ‘seeds.csv’ saved [8973/8973]



In [38]:
df2 = spark.read.csv("seeds.csv", header=True, inferSchema=True)

In [39]:
df2.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length of kernel: double (nullable = true)
 |-- width of kernel: double (nullable = true)
 |-- asymmetry coefficient: double (nullable = true)
 |-- length of kernel groove: double (nullable = true)



In [41]:
df2.show(1)

+-----+---------+-----------+----------------+---------------+---------------------+-----------------------+
| area|perimeter|compactness|length of kernel|width of kernel|asymmetry coefficient|length of kernel groove|
+-----+---------+-----------+----------------+---------------+---------------------+-----------------------+
|15.26|    14.84|      0.871|           5.763|          3.312|                2.221|                   5.22|
+-----+---------+-----------+----------------+---------------+---------------------+-----------------------+
only showing top 1 row


In [45]:
assembler = VectorAssembler(inputCols=["area","perimeter","compactness","length of kernel","width of kernel","asymmetry coefficient","length of kernel groove"], outputCol="features")
df2_new = assembler.transform(df2)

In [46]:
df2_new.show(5)

+-----+---------+-----------+----------------+---------------+---------------------+-----------------------+--------------------+
| area|perimeter|compactness|length of kernel|width of kernel|asymmetry coefficient|length of kernel groove|            features|
+-----+---------+-----------+----------------+---------------+---------------------+-----------------------+--------------------+
|15.26|    14.84|      0.871|           5.763|          3.312|                2.221|                   5.22|[15.26,14.84,0.87...|
|14.88|    14.57|     0.8811|           5.554|          3.333|                1.018|                  4.956|[14.88,14.57,0.88...|
|14.29|    14.09|      0.905|           5.291|          3.337|                2.699|                  4.825|[14.29,14.09,0.90...|
|13.84|    13.94|     0.8955|           5.324|          3.379|                2.259|                  4.805|[13.84,13.94,0.89...|
|16.14|    14.99|     0.9034|           5.658|          3.562|                1.355|      

In [47]:
kmeans = KMeans(k=7)
model = kmeans.fit(df2_new)

In [48]:
predictions = model.transform(df2_new)

In [49]:
predictions.show()

+-----+---------+-----------+----------------+---------------+---------------------+-----------------------+--------------------+----------+
| area|perimeter|compactness|length of kernel|width of kernel|asymmetry coefficient|length of kernel groove|            features|prediction|
+-----+---------+-----------+----------------+---------------+---------------------+-----------------------+--------------------+----------+
|15.26|    14.84|      0.871|           5.763|          3.312|                2.221|                   5.22|[15.26,14.84,0.87...|         5|
|14.88|    14.57|     0.8811|           5.554|          3.333|                1.018|                  4.956|[14.88,14.57,0.88...|         5|
|14.29|    14.09|      0.905|           5.291|          3.337|                2.699|                  4.825|[14.29,14.09,0.90...|         1|
|13.84|    13.94|     0.8955|           5.324|          3.379|                2.259|                  4.805|[13.84,13.94,0.89...|         1|
|16.14|    14

In [51]:
predictions.groupby('prediction').count().orderBy('prediction').show()

+----------+-----+
|prediction|count|
+----------+-----+
|         0|   21|
|         1|   41|
|         2|   33|
|         3|   24|
|         4|   48|
|         5|   28|
|         6|   15|
+----------+-----+



Tweak the number of clusters from time to time